# 80. ID番号への変換

問題51で構築した学習データ中の単語にユニークなID番号を付与したい．学習データ中で最も頻出する単語に1，2番目に頻出する単語に2，……といった方法で，学習データ中で2回以上出現する単語にID番号を付与せよ．そして，与えられた単語列に対して，ID番号の列を返す関数を実装せよ．ただし，出現頻度が2回未満の単語のID番号はすべて0とせよ．

In [70]:
from collections import Counter
import pickle
import ast
import torch.nn.utils.rnn as rnn
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm


In [71]:
#  単語とidの辞書を作成

word_2_id = {}
words = []

with open('data/train.txt') as fi:
    for line in fi:
        sent = line.split('\t')[0]
        sent = sent.replace("'","").replace('"','').replace('.','').replace('?','')
        for word in sent.split():
            words.append(word)#  出現する単語をとりあえず入れとく
        
#  単語の数を数える
word_count = Counter(words) 
#  出現回数順にソート
sort_word_count = word_count.most_common()
#print(sort_word_count)
#  一応保存
with open('work/word_count.pickle', mode='wb') as f:
    pickle.dump(sort_word_count, f)


#  idを付与
for i, w_and_c in enumerate(sort_word_count):
    word = w_and_c[0]
    count = w_and_c[1]
    if count > 1:
        word_2_id[word] = i + 1

In [72]:
#  word_2_idを利用してidを付与する関数
def text_2_id_list(text):
    sentence_id_list = []
    words = text.replace("'","").replace('"','').replace('.','').replace('?','')
    words = words.rstrip().split()
    for word in words:
        if word in word_2_id:
            word_id = word_2_id[word]
            sentence_id_list.append(word_id)
        else:
            sentence_id_list.append(int(0))
            
    return sentence_id_list
        

In [1170]:
sample = 'I want to go to Japan.'
text_2_id_list(sample)

[104, 2002, 1, 1533, 1, 301]

In [1171]:
print(len(word_2_id))
print(len(word_3_id))

10137
10137


In [1158]:
word_3_id = {}
myfile = open('work/word_count.pickle', 'rb')
rmydata = pickle.load(myfile)
for i, w_and_c in enumerate(rmydata):
    word = w_and_c[0]
    
    count = w_and_c[1]
    if count > 1:
        word_3_id[word] = i + 1



In [73]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import numpy as np
import matplotlib.pyplot as plt
'''GPUチェック'''
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


cpu


In [898]:
#ハイパラたち

#  単語埋め込み次元数
dw = 300
#  隠れ状態ベクトルの次元数
dh = 50

input_size = len(words) + 2 #語彙サイズ＋１尚且つ０のやつもあるので＋１
padding_idx = 0
emb_size = dw
hidden_size = dh
output_size = 4

In [899]:
print(model)

RNN(
  (embedding): Embedding(109762, 300, padding_idx=109761)
  (rnn): RNN(300, 50, batch_first=True)
  (fc): Linear(in_features=50, out_features=4, bias=True)
)


In [900]:
#RNNを用いたモデル定義を行う

class My_rnn(nn.Module):
    def __init__(self):
        super().__init__()
        #  埋め込みの定義
        self.emb = nn.Embedding(input_size, emb_size, padding_idx)
        #  RNNの定義
        self.rnn = nn.RNN(emb_size, hidden_size, batch_first=True)#batch_size, seq_len, dim
        #  出力層の定義
        self.linear = nn.Linear(hidden_size, output_size)
        #  softmax層
        #self.softmax = nn.Softmax(dim=1)
        
    #  予測関数の定義    
    def forward(self, x, h0=None):
        x = self.emb(x)#  埋め込み層
        y, h = self.rnn(x, h0)#  xと初期の隠れ状態h0でRNNモデルを実行
        y = y[:, -1, :]#  時間経過の中で最後のステップを取得  #系列長、バッチサイズ、次元数
        y = self.fc(y)#  全結合層 →　４次元へ
        #y = self.softmax(y, dim=1)#  ４次元を確率へ
        return y



In [901]:
print(model)

RNN(
  (embedding): Embedding(109762, 300, padding_idx=109761)
  (rnn): RNN(300, 50, batch_first=True)
  (fc): Linear(in_features=50, out_features=4, bias=True)
)


In [69]:
sample = 'I want to go to Japan.'

x = torch.tensor(text_2_id_list(sample), dtype=torch.int64)
print(x)
print(x.size())
print(model(x))
print(F.softmax(model(x), dim=-1))
print(torch.argmax(model(x)))

NameError: name 'text_2_id_list' is not defined

#  82. 確率的勾配降下法による学習

In [966]:
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [967]:
#ハイパラたち

#  単語埋め込み次元数
dw = 300
#  隠れ状態ベクトルの次元数
dh = 50

input_size = len(words) + 2 #語彙サイズ＋１尚且つ０のやつもあるので＋１,"-1"の分で＋１
padding_idx = len(words) +1
emb_size = dw
hidden_size = dh
output_size = 4

In [74]:
#ハイパラたち

#  単語埋め込み次元数
dw = 300
#  隠れ状態ベクトルの次元数
dh = 50

input_size = len(word_2_id.values()) + 2 #語彙サイズ＋１尚且つ０のやつもあるので＋１,"-1"の分で＋１
padding_idx = len(word_2_id.values()) +1
emb_size = dw
hidden_size = dh
output_size = 4

In [1230]:
print(input_size)
print(padding_idx)

10139
10138


In [1231]:
class My_rnn(nn.Module):
    def __init__(self):
        super().__init__()
        #  埋め込みの定義
        self.emb = nn.Embedding(input_size, emb_size, padding_idx)
        #  RNNの定義
        self.rnn = nn.RNN(emb_size, hidden_size,batch_first=True)#batch_size, seq_len, dim
        #  出力層の定義
        self.linear = nn.Linear(hidden_size, output_size)
        #  softmax層
        #self.softmax = nn.Softmax(dim=1)
        
    #  予測関数の定義    
    def forward(self, x, h0=None):
        x = self.emb(x)#  埋め込み層
        
        y, h = self.rnn(x, h0)#  xと初期の隠れ状態h0でRNNモデルを実行
        #print(y.size())
        y = y[:, -1, :]#  時間経過の中で最後のステップを取得  #系列長、バッチサイズ、次元数
        y = self.linear(y)#  全結合層 →　４次元へ
        #y = self.softmax(y, dim=1)#  ４次元を確率へ
        return y



In [1232]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data_path):
        # データの読み出し
        dataframe = []
        with open(data_path) as f:
            for line in f:
                sent = line.split('\t')
                dataframe.append(sent)
        self.dataframe = dataframe
    # データのサイズ
    def __len__(self):
        return len(self.dataframe)
    # データとラベルの取得
    def __getitem__(self, idx):
        category_dict = {'b': 0, 't': 1, 'e':2, 'm':3}
        try:
            label = category_dict[self.dataframe[idx][1].rstrip()]
        except:
            print(self.dataframe[idx][1].rstrip())
        #print(label)
        #print(self.dataframe[idx][0])
        text_id = torch.tensor(text_2_id_list(self.dataframe[idx][0]), dtype=torch.int64)
        # text_id, labelの順でリターン
        return text_id, label

In [1233]:
def pad_sec(batch):
    batch_tensor = [item[0] for item in batch]
    labels = torch.LongTensor([item[1] for item in batch])
    pad_tensors = rnn.pad_sequence(batch_tensor,batch_first=True, padding_value=padding_idx)
    return pad_tensors, labels

In [1234]:
batch_size=50
train_dataset = MyDataset('data/train.txt')
valid_dataset = MyDataset('data/valid.txt')
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=pad_sec)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, collate_fn=pad_sec)

In [1235]:
#  学習用の関数を定義



def training(num_epochs):
    model = My_rnn()
    train_avg_loss = []
    test_avg_loss = []
    train_acc = []
    test_acc = []
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001)
    model.train()

    for epoch in tqdm(range(num_epochs)):
        total_train_loss = 0
        total_test_loss = 0  
        train_pred_y_list = []
        true_y_list = []
        
        for x_train_tensor, y_label in train_dataloader:
            optimizer.zero_grad()
            pred_y = model(x_train_tensor)
            loss = criterion(pred_y, y_label)
            loss.backward()
            optimizer.step()
            #予想したラベルを貯めておく
            train_pred_y = torch.argmax(pred_y, dim=1)
            train_pred_y_list.append(train_pred_y)
            true_y_list.append(y_label)
            
            total_train_loss = total_train_loss + loss.item()
            
        
        #  logの表示
        print(f'epoch:{epoch}')
        avg_loss = total_train_loss / len(train_pred_y_list)
        train_avg_loss.append(avg_loss)
        #print(pred_y)
        #print(train_pred_y)
        #print(train_pred_y_list[0])
        #print(true_y_list[0])
        train_y_pre = torch.cat(train_pred_y_list, dim = 0)
        true_y_tensor = torch.cat(true_y_list, dim = 0)
        
        acc_score = accuracy_score(train_y_pre, true_y_tensor)
        train_acc.append(acc_score)
        print(f'acc:{acc_score}')
        print(f'loss:{avg_loss}')
        
        
    #print("train_loss", train_avg_loss)
    #print("train_acc", train_acc)
            

In [1236]:
#for x, y in train_dataloader:
    #print(x.size())

In [1239]:
print(My_rnn())

My_rnn(
  (emb): Embedding(10139, 300, padding_idx=10138)
  (rnn): RNN(300, 50, batch_first=True)
  (linear): Linear(in_features=50, out_features=4, bias=True)
)


In [1240]:
training(10)

 10%|█████████▎                                                                                   | 1/10 [00:01<00:14,  1.56s/it]

epoch:0
acc:0.39189442156495696
loss:1.3647306711874276


 20%|██████████████████▌                                                                          | 2/10 [00:02<00:11,  1.49s/it]

epoch:1
acc:0.4376637963309622
loss:1.3129034231756336


 30%|███████████████████████████▉                                                                 | 3/10 [00:04<00:10,  1.46s/it]

...

epoch:7
acc:0.4476787719955073
loss:1.176461632563689


 90%|███████████████████████████████████████████████████████████████████████████████████▋         | 9/10 [00:13<00:01,  1.44s/it]

epoch:8
acc:0.4487083489329839
loss:1.1715686326829073


100%|████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:14<00:00,  1.45s/it]

epoch:9
acc:0.4517970797454137
loss:1.1683023594250188


# 83. ミニバッチ化・GPU上での学習

In [ ]:
def training(num_epochs):
    model = My_rnn()
    train_avg_loss = []
    test_avg_loss = []
    train_acc = []
    test_acc = []
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001)
    model.train()
    
    # 初期設定
    # GPUが使えるか確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(torch.cuda.get_device_name())
    print("使用デバイス:", device)
    
    model.to(device)
    
    for epoch in tqdm(range(num_epochs)):
        total_train_loss = 0
        total_test_loss = 0  
        train_pred_y_list = []
        true_y_list = []
        
        for x_train_tensor, y_label in train_dataloader:
            x_train_tensor = x_train_tensor.to(device)
            y_label = y_label.to(device)
            optimizer.zero_grad()
            pred_y = model(x_train_tensor)
            loss = criterion(pred_y, y_label)
            loss.backward()
            optimizer.step()
            #予想したラベルを貯めておく
            train_pred_y = torch.argmax(pred_y, dim=1)
            train_pred_y_list.append(train_pred_y)
            true_y_list.append(y_label)
            
            total_train_loss = total_train_loss + loss.item()
            
        
        #  logの表示
        print(f'epoch:{epoch}')
        avg_loss = total_train_loss / len(train_pred_y_list)
        train_avg_loss.append(avg_loss)
        #print(pred_y)
        #print(train_pred_y)
        #print(train_pred_y_list[0])
        #print(true_y_list[0])
        train_y_pre = torch.cat(train_pred_y_list, dim = 0)
        true_y_tensor = torch.cat(true_y_list, dim = 0)
        
        acc_score = accuracy_score(train_y_pre, true_y_tensor)
        train_acc.append(acc_score)
        print(f'acc:{acc_score}')
        print(f'loss:{avg_loss}')

batch_size=50
num_epoch = 10
NVIDIA GeForce GTX 1080 Ti
使用デバイス: cuda:0
  0%|                                                                                                     | 0/10 [00:00<?, ?it/s]
epoch:0
acc:0.22117184575065518
loss:1.3987760009052597
 10%|█████████▎                                                                                   | 1/10 [00:00<00:08,  1.10it/s]
epoch:1
acc:0.44009734181954324
loss:1.334488627509536
 20%|██████████████████▌                                                                          | 2/10 [00:01<00:07,  1.09it/s]
epoch:2
acc:0.44721078247847246
loss:1.2850932400917339
 30%|███████████████████████████▉                                                                 | 3/10 [00:02<00:06,  1.08it/s]
epoch:3
acc:0.4477723698989143
loss:1.2465784989784812
 40%|█████████████████████████████████████▏                                                       | 4/10 [00:03<00:05,  1.07it/s]
epoch:4
acc:0.44833395731935605
loss:1.2178469194430057
 50%|██████████████████████████████████████████████▌                                              | 5/10 [00:04<00:04,  1.07it/s]
epoch:5
acc:0.44926993635342566
loss:1.1977538945518922
 60%|███████████████████████████████████████████████████████▊                                     | 6/10 [00:05<00:03,  1.07it/s]
epoch:6
acc:0.4498315237738675
loss:1.1844539280249693
 70%|█████████████████████████████████████████████████████████████████                            | 7/10 [00:06<00:02,  1.07it/s]
epoch:7
acc:0.45273305877948333
loss:1.1759229227204189
 80%|██████████████████████████████████████████████████████████████████████████▍                  | 8/10 [00:07<00:01,  1.07it/s]
epoch:8
acc:0.45479221265443653
loss:1.1704850436371064
 90%|███████████████████████████████████████████████████████████████████████████████████▋         | 9/10 [00:08<00:00,  1.06it/s]
epoch:9
acc:0.4559153874953201
loss:1.1669585863563503
100%|████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:09<00:00,  1.07it/s

#  84. 単語ベクトルの導入

事前学習済みの単語ベクトル（例えば，Google Newsデータセット（約1,000億単語）での学習済み単語ベクトル）で単語埋め込みemb(x)
を初期化し，学習せよ．

In [1174]:
trainee_n-miura/chapter07/data/GoogleNews-vectors-negative300.bin

zsh:1: no matches found: https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing


In [2]:
from gensim.models import KeyedVectors
import gensim


# 学習済みモデルのロード
key_model = gensim.models.KeyedVectors.load_word2vec_format("/Users/miuranaoki/nlp100/100knock-2023/trainee_n-miura/chapter07/data/GoogleNews-vectors-negative300.bin", binary=True)

In [3]:
#ハイパラたち

#  単語埋め込み次元数
dw = 300
#  隠れ状態ベクトルの次元数
dh = 50

input_size = len(word_2_id.values()) + 2 #語彙サイズ＋１尚且つ０のやつもあるので＋１,"-1"の分で＋１
padding_idx = len(word_2_id.values()) +1
emb_size = dw
hidden_size = dh
output_size = 4

NameError: name 'word_2_id' is not defined

In [ ]:
syoki = np.zeros((input_size,300))

In [ ]:
syoki.shape

In [1219]:
model["apple"].shape

(300,)

In [22]:
c = 0
for w, i in word_2_id.items():
    try:
        syoki[i] = key_model[w]
        c += 1
        #print(w)
    except:
        syoki[i] = np.random.normal(scale=1.0, size= (300, ))

In [23]:
print(f'{c}/{input_size}')
weights = torch.from_numpy(syoki.astype((np.float32)))

8701/10139


In [24]:
weights.size()

torch.Size([10139, 300])

In [25]:
with open('work/weights.pickle', mode='wb') as f:
    pickle.dump(weights, f)

In [26]:
with open('work/weights.pickle','rb') as fi:
    weight = pickle.load(fi)

In [27]:
print(weight.size())

torch.Size([10139, 300])


In [1264]:
class My_rnn(nn.Module):
    def __init__(self):
        super().__init__()
        #  埋め込みの定義
        self.emb = nn.Embedding.from_pretrained(wei, padding_idx)
        #  RNNの定義
        self.rnn = nn.RNN(emb_size, hidden_size,batch_first=True)#batch_size, seq_len, dim
        #  出力層の定義
        self.linear = nn.Linear(hidden_size, output_size)
        #  softmax層
        #self.softmax = nn.Softmax(dim=1)
        
    #  予測関数の定義    
    def forward(self, x, h0=None):
        x = self.emb(x)#  埋め込み層
        
        y, h = self.rnn(x, h0)#  xと初期の隠れ状態h0でRNNモデルを実行
        #print(y.size())
        y = y[:, -1, :]#  時間経過の中で最後のステップを取得  #系列長、バッチサイズ、次元数
        y = self.linear(y)#  全結合層 →　４次元へ
        #y = self.softmax(y, dim=1)#  ４次元を確率へ
        return y



In [1265]:
#  学習用の関数を定義



def training(num_epochs):
    model = My_rnn()
    train_avg_loss = []
    test_avg_loss = []
    train_acc = []
    test_acc = []
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001)
    model.train()

    for epoch in tqdm(range(num_epochs)):
        total_train_loss = 0
        total_test_loss = 0  
        train_pred_y_list = []
        true_y_list = []
        
        for x_train_tensor, y_label in train_dataloader:
            optimizer.zero_grad()
            pred_y = model(x_train_tensor)
            loss = criterion(pred_y, y_label)
            loss.backward()
            optimizer.step()
            #予想したラベルを貯めておく
            train_pred_y = torch.argmax(pred_y, dim=1)
            train_pred_y_list.append(train_pred_y)
            true_y_list.append(y_label)
            
            total_train_loss = total_train_loss + loss.item()
            
        
        #  logの表示
        print(f'epoch:{epoch}')
        avg_loss = total_train_loss / len(train_pred_y_list)
        train_avg_loss.append(avg_loss)
        #print(pred_y)
        #print(train_pred_y)
        #print(train_pred_y_list[0])
        #print(true_y_list[0])
        train_y_pre = torch.cat(train_pred_y_list, dim = 0)
        true_y_tensor = torch.cat(true_y_list, dim = 0)
        
        acc_score = accuracy_score(train_y_pre, true_y_tensor)
        train_acc.append(acc_score)
        print(f'acc:{acc_score}')
        print(f'loss:{avg_loss}')
        

In [1273]:
len(word_2_id.values()) +1

10138

batch_size=50
num_epoch = 10
NVIDIA GeForce GTX 1080 Ti
使用デバイス: cuda:0
  0%|                                                                                                                                                  | 0/10 [00:00<?, ?it/s]
epoch:0
acc:0.4190378135529764
loss:1.3555285496132397
 10%|█████████████▊                                                                                                                            | 1/10 [00:00<00:06,  1.42it/s]
epoch:1
acc:0.42250093597903404
loss:1.3046491497030883
 20%|███████████████████████████▌                                                                                                              | 2/10 [00:01<00:05,  1.43it/s]
epoch:2
acc:0.422688131785848
loss:1.2655016122577347
 30%|█████████████████████████████████████████▍                                                                                                | 3/10 [00:02<00:04,  1.43it/s]
epoch:3
acc:0.4232497192062898
loss:1.2350091828364078
 40%|███████████████████████████████████████████████████████▏                                                                                  | 4/10 [00:02<00:04,  1.44it/s]
epoch:4
acc:0.42381130662673155
loss:1.2117288095928798
 50%|█████████████████████████████████████████████████████████████████████                                                                     | 5/10 [00:03<00:03,  1.45it/s]
epoch:5
acc:0.4254960688880569
loss:1.194669687859366
 60%|██████████████████████████████████████████████████████████████████████████████████▊                                                       | 6/10 [00:04<00:02,  1.45it/s]
epoch:6
acc:0.42792961437663796
loss:1.1827999715493105
 70%|████████████████████████████████████████████████████████████████████████████████████████████████▌                                         | 7/10 [00:04<00:02,  1.44it/s]
epoch:7
acc:0.4309247472856608
loss:1.1749619407074474
 80%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                           | 8/10 [00:05<00:01,  1.45it/s]
epoch:8
acc:0.43345189067764883
loss:1.1700092726778761
 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏             | 9/10 [00:06<00:00,  1.45it/s]
epoch:9
acc:0.43569824035941596
loss:1.1669701131704813

#  85. 双方向RNN・多層化

In [28]:
class My_bi_rnn(nn.Module):
    def __init__(self):
        super().__init__()
        #  埋め込みの定義
        self.emb = nn.Embedding.from_pretrained(weight, padding_idx)
        #  RNNの定義
        self.rnn = nn.RNN(emb_size, hidden_size,batch_first=True,bidirectional=True,num_layers=3)#batch_size, seq_len, dim
        #  出力層の定義
        self.linear = nn.Linear(hidden_size * 2, output_size)

        #  softmax層
        #self.softmax = nn.Softmax(dim=1)
        
    #  予測関数の定義    
    def forward(self, x, h0=None):
        x = self.emb(x)#  埋め込み層
        
        y, h = self.rnn(x, h0)#  xと初期の隠れ状態h0でRNNモデルを実行
        #print(y.size())
        y = y[:, -1, :]#  時間経過の中で最後のステップを取得  #系列長、バッチサイズ、次元数
        y = self.linear(y)#  全結合層 →　４次元へ
        #y = self.softmax(y, dim=1)#  ４次元を確率へ
        return y

In [30]:
model = My_bi_rnn()
print(model)

My_bi_rnn(
  (emb): Embedding(10139, 300)
  (rnn): RNN(300, 50, num_layers=3, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=100, out_features=4, bias=True)
)


epoch:0
acc:0.3379820292025459
loss:1.3512836616730022
 10%|█████████████▊                                                                                                                            | 1/10 [00:00<00:08,  1.12it/s]
epoch:1
acc:0.4280232122800449
loss:1.2448863080728834
 20%|███████████████████████████▌                                                                                                              | 2/10 [00:01<00:07,  1.12it/s]
epoch:2
acc:0.42933358292774243
loss:1.19530474749681
 30%|█████████████████████████████████████████▍                                                                                                | 3/10 [00:02<00:06,  1.13it/s]
epoch:3
acc:0.4324223137401722
loss:1.1752584732581521
 40%|███████████████████████████████████████████████████████▏                                                                                  | 4/10 [00:03<00:05,  1.13it/s]
epoch:4
acc:0.43775739423436916
loss:1.167796624598102
 50%|█████████████████████████████████████████████████████████████████████                                                                     | 5/10 [00:04<00:04,  1.15it/s]
epoch:5
acc:0.4404717334331711
loss:1.1649580748281747
 60%|██████████████████████████████████████████████████████████████████████████████████▊                                                       | 6/10 [00:05<00:03,  1.18it/s]
epoch:6
acc:0.44122051666042683
loss:1.1637549971308663
 70%|████████████████████████████████████████████████████████████████████████████████████████████████▌                                         | 7/10 [00:06<00:02,  1.18it/s]
epoch:7
acc:0.4425308873081243
loss:1.1631508413876328
 80%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                           | 8/10 [00:06<00:01,  1.17it/s]
epoch:8
acc:0.4440284537626357
loss:1.1627824426254378
 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏             | 9/10 [00:07<00:00,  1.21it/s]
epoch:9
acc:0.4457132160239611
loss:1.1625143320203941
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:08<00:00,  1.17it/s]

#  86. 畳み込みニューラルネットワーク (CNN)

In [169]:
# 第１引数はインプットのチャネル（今回は1）を指定
# 自然言語処理で畳み込む場合、異なる単語分散表現（word2vecとfasttextみたいな）などを使って、
# 複数チャネルとみなす方法もあるようです。
# 第２引数はアウトプットのチャネル数で、今回は同じフィルターを2枚畳み込みたいので、2を指定
# カーネルサイズは高さ×幅を指定しており、幅は図で説明した通り、単語ベクトルの次元数5を指定



dw = 300
#  隠れ状態ベクトルの次元数
dh = 50

input_size = len(word_2_id.values()) + 2 #語彙サイズ＋１尚且つ０のやつもあるので＋１,"-1"の分で＋１
padding_idx = len(word_2_id.values()) +1
emb_size = dw
hidden_size = dh
output_size = 4

out_channels = 100
kernel_size = 3
stride = 1
padding = 1


In [170]:
with open('work/weights.pickle','rb') as fi:
    weight = pickle.load(fi)

In [267]:
class My_cnn(nn.Module):
    def __init__(self):
        super().__init__()
        #  埋め込みの定義
        self.emb = nn.Embedding.from_pretrained(weight, padding_idx)
        #  CNNの定義
        self.conv = nn.Conv1d(dw, dh, kernel_size, stride, padding)# in_channels:dw, out_channels: dh
        #  出力層の定義
        self.linear = nn.Linear(dh, output_size)

        
    #  予測関数の定義    
    def forward(self, x):
        # x = [batch, 系列長]
        x = self.emb(x)#  埋め込み層 torch.Size([6(系列長), 300])
        x = x.unsqueeze(1)#  torch.Size([6, 300]) -> torch.Size([6, 300, 1])
        #print(x.size())
        x = x.view(x.shape[1], x.shape[2], x.shape[0])#  torch.Size([6, 300, 1]) -> torch.Size([1, 300, 6])
        #print(x.size())
        conv = self.conv(x)#  torch.Size([1, 300, 6]) -> torch.Size([1, 50, 6])
        #print(conv.size())
        x = F.relu(conv)
        x = F.max_pool1d(x, x.size()[2])#  torch.Size([1, 50, 6]) -> torch.Size([1, 50, 1])
        #print(x.size())
        x = x.view(x.shape[0], x.shape[1])#  torch.Size([1, 50, 1]) -> torch.Size([1, 50])
        #print(x.size())
        #print(x[:, :, -1])
        y = self.linear(x) # torch.Size([1, 50]) -> torch.Size([1, 4])
        return y


In [268]:
sample = 'I want to go to Japan.'
x = torch.tensor(text_2_id_list(sample))
model = My_cnn()
print(x)
print(x.size())

print(nn.Softmax(dim=-1)(model(x)))

tensor([ 104, 2002,    1, 1533,    1,  301])
torch.Size([6])
tensor([[0.2544, 0.2197, 0.2197, 0.3062]], grad_fn=<SoftmaxBackward0>)


#  87. 確率的勾配降下法によるCNNの学習

確率的勾配降下法（SGD: Stochastic Gradient Descent）を用いて，問題86で構築したモデルを学習せよ．訓練データ上の損失と正解率，評価データ上の損失と正解率を表示しながらモデルを学習し，適当な基準（例えば10エポックなど）で終了させよ．

In [ ]:
class My_cnn(nn.Module):
    def __init__(self):
        super().__init__()
        #  埋め込みの定義
        self.emb = nn.Embedding.from_pretrained(weights, padding_idx)
        #  CNNの定義
        self.conv = nn.Conv1d(dw, dh, kernel_size, stride, padding)# in_channels:dw, out_channels: dh
        #  出力層の定義
        self.linear = nn.Linear(dh, output_size)

        
    #  予測関数の定義    
    def forward(self, x):
        #print(x.size()) torch.Size([50, 15])
        x = self.emb(x)#  埋め込み層 
        #print(x.size()) torch.Size([50, 15, 300])
        x = x.view(x.shape[0], x.shape[2], x.shape[1])
        #print(x.size()) #torch.Size([50, 300, 15])
        conv = self.conv(x)
        #print(conv.size()) # torch.Size([50, 50, 15])
        x = F.relu(conv)
        x = F.max_pool1d(x, x.size()[2])
        #print(x.size()) # torch.Size([50, 50, 1])
        x = x.view(x.shape[0], x.shape[1])
        #print(x.size()) #  torch.Size([50, 50])
        #print(x[:, :, -1])
        y = self.linear(x) 
        #print(y.size()) # torch.Size([50, 4])
        return y

In [ ]:
def training_and_valid(num_epochs):
    model = My_cnn()
    train_avg_loss = []
    valid_avg_loss = []
    train_acc = []
    valid_acc = []
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001)
    model.train()

    # 初期設定
    # GPUが使えるか確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(torch.cuda.get_device_name())
    print("使用デバイス:", device)

    model.to(device)

    for epoch in tqdm(range(num_epochs)):
        total_train_loss = 0
        total_valid_loss = 0 
        valid_loss = 0
        correct_valid = 0 
        train_pred_y_list = []
        valid_pred_y_list = []
        true_y_list = []
        valid_true_y_list = []
        
        for x_train_tensor, y_label in train_dataloader:
            x_train_tensor = x_train_tensor.to(device)
            y_label = y_label.to(device)
            optimizer.zero_grad()
            pred_y = model(x_train_tensor)
            #print(pred_y.size())
            #print(y_label.size())
            loss = criterion(pred_y, y_label)
            loss.backward()
            optimizer.step()
            #予想したラベルを貯めておく
            train_pred_y = torch.argmax(pred_y, dim=1)
            train_pred_y_list.append(train_pred_y)
            true_y_list.append(y_label)
            
            total_train_loss = total_train_loss + loss.item()


        model.eval()
        with torch.no_grad(): 
            for x_valid_tensor, y_valid_label in valid_dataloader:
                x_valid_tensor = x_valid_tensor.to(device)
                y_valid_label = y_valid_label.to(device)
                valid_pred_y = model(x_valid_tensor)
                loss = criterion(valid_pred_y, y_valid_label)
                total_valid_loss += loss.item()
                #予想したラベルを貯めておく
                valid_pred_y = torch.argmax(valid_pred_y, dim=1)
                valid_pred_y_list.append(valid_pred_y)
                valid_true_y_list.append(y_valid_label)

        #  logの表示
        print(f'epoch:{epoch}')
        tra_avg_loss = total_train_loss / len(train_pred_y_list)
        val_avg_loss = total_valid_loss / len(valid_pred_y_list)
        train_avg_loss.append(tra_avg_loss)
        valid_avg_loss.append(val_avg_loss)

        train_y_pre = torch.cat(train_pred_y_list, dim = 0)
        valid_y_pre = torch.cat(valid_pred_y_list, dim = 0)

        true_y_tensor = torch.cat(true_y_list, dim = 0)
        valid_true_y_tensor = torch.cat(valid_true_y_list, dim = 0)

        train_y_pre = train_y_pre.cpu()
        valid_y_pre = valid_y_pre.cpu()

        true_y_tensor = true_y_tensor.cpu()
        valid_true_y_tensor = valid_true_y_tensor.cpu()

        acc_score = accuracy_score(train_y_pre, true_y_tensor)
        valid_acc_score = accuracy_score(valid_y_pre, valid_true_y_tensor)

        train_acc.append(acc_score)
        valid_acc.append(valid_acc_score)

        print(f'train_acc:{acc_score}')
        print(f'valid_acc:{valid_acc_score}')

        print(f'train_avg_loss:{statistics.mean(train_avg_loss)}')
        print(f'valid_avg_loss:{statistics.mean(valid_avg_loss)}')

            
            
            



num_epoch = 5
batch_size = 10

#  main
if __name__ == '__main__':
    print(f'batch_size = {batch_size}')
    print(f'num_epoch = {num_epoch}')

    training_and_valid(num_epoch)

 % python q87.py
batch_size = 10
num_epoch = 5
NVIDIA GeForce GTX 1080 Ti
使用デバイス: cuda:0
  0%|                                                                                       | 0/5 [00:00<?, ?it/s]
epoch:0
train_acc:0.37411081991763384
valid_acc:0.4865269461077844
train_avg_loss:1.3224773526303226
valid_avg_loss:1.189154571561671
 20%|███████████████▊                                                               | 1/5 [00:02<00:10,  2.69s/it]
epoch:1
train_acc:0.484462748034444
valid_acc:0.49850299401197606
train_avg_loss:1.2469308870322688
valid_avg_loss:1.1653455655966232
 40%|███████████████████████████████▌                                               | 2/5 [00:05<00:07,  2.47s/it]
epoch:2
train_acc:0.4928865593410708
valid_acc:0.5217065868263473
train_avg_loss:1.21415727566736
valid_avg_loss:1.1525217235384888
 60%|███████████████████████████████████████████████▍                               | 3/5 [00:07<00:04,  2.37s/it]
epoch:3
train_acc:0.510108573567952
valid_acc:0.5284431137724551
train_avg_loss:1.1947822521037315
valid_avg_loss:1.1436533074992807
 80%|███████████████████████████████████████████████████████████████▏               | 4/5 [00:09<00:02,  2.34s/it]
epoch:4
train_acc:0.5203107450393111
valid_acc:0.5276946107784432
train_avg_loss:1.1810466110316027
valid_avg_loss:1.1365581802467801
100%|███████████████████████████████████████████████████████████████████████████████| 5/5 [00:11<00:00,  2.37s/it]

#  88. パラメータチューニング

問題85や問題87のコードを改変し，ニューラルネットワークの形状やハイパーパラメータを調整しながら，高性能なカテゴリ分類器を構築せよ．



In [ ]:
def training_and_valid(num_epochs):
    model = My_cnn()
    train_avg_loss = []
    valid_avg_loss = []
    train_acc = []
    valid_acc = []
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())
    model.train()

    # 初期設定
    # GPUが使えるか確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(torch.cuda.get_device_name())
    print("使用デバイス:", device)

    model.to(device)

    for epoch in tqdm(range(num_epochs)):
        total_train_loss = 0
        total_valid_loss = 0 
        valid_loss = 0
        correct_valid = 0 
        train_pred_y_list = []
        valid_pred_y_list = []
        true_y_list = []
        valid_true_y_list = []
        
        for x_train_tensor, y_label in train_dataloader:
            x_train_tensor = x_train_tensor.to(device)
            y_label = y_label.to(device)
            optimizer.zero_grad()
            pred_y = model(x_train_tensor)
            #print(pred_y.size())
            #print(y_label.size())
            loss = criterion(pred_y, y_label)
            loss.backward()
            optimizer.step()
            #予想したラベルを貯めておく
            train_pred_y = torch.argmax(pred_y, dim=1)
            train_pred_y_list.append(train_pred_y)
            true_y_list.append(y_label)
            
            total_train_loss = total_train_loss + loss.item()


        model.eval()
        with torch.no_grad(): 
            for x_valid_tensor, y_valid_label in valid_dataloader:
                x_valid_tensor = x_valid_tensor.to(device)
                y_valid_label = y_valid_label.to(device)
                valid_pred_y = model(x_valid_tensor)
                loss = criterion(valid_pred_y, y_valid_label)
                total_valid_loss += loss.item()
                #予想したラベルを貯めておく
                valid_pred_y = torch.argmax(valid_pred_y, dim=1)
                valid_pred_y_list.append(valid_pred_y)
                valid_true_y_list.append(y_valid_label)

        #  logの表示
        print(f'epoch:{epoch}')
        tra_avg_loss = total_train_loss / len(train_pred_y_list)
        val_avg_loss = total_valid_loss / len(valid_pred_y_list)
        train_avg_loss.append(tra_avg_loss)
        valid_avg_loss.append(val_avg_loss)

        train_y_pre = torch.cat(train_pred_y_list, dim = 0)
        valid_y_pre = torch.cat(valid_pred_y_list, dim = 0)

        true_y_tensor = torch.cat(true_y_list, dim = 0)
        valid_true_y_tensor = torch.cat(valid_true_y_list, dim = 0)

        train_y_pre = train_y_pre.cpu()
        valid_y_pre = valid_y_pre.cpu()

        true_y_tensor = true_y_tensor.cpu()
        valid_true_y_tensor = valid_true_y_tensor.cpu()

        acc_score = accuracy_score(train_y_pre, true_y_tensor)
        valid_acc_score = accuracy_score(valid_y_pre, valid_true_y_tensor)

        train_acc.append(acc_score)
        valid_acc.append(valid_acc_score)

        print(f'train_acc:{acc_score}')
        print(f'valid_acc:{valid_acc_score}')

        print(f'train_avg_loss:{statistics.mean(train_avg_loss)}')
        print(f'valid_avg_loss:{statistics.mean(valid_avg_loss)}')

            
            
            



num_epoch = 5
batch_size = 10

#  main
if __name__ == '__main__':
    print(f'batch_size = {batch_size}')
    print(f'num_epoch = {num_epoch}')

    training_and_valid(num_epoch)

In [ ]:
% python q88.py
batch_size = 10
num_epoch = 5
NVIDIA GeForce GTX 1080 Ti
使用デバイス: cuda:0
  0%|                                                                                       | 0/5 [00:00<?, ?it/s]
epoch:0
train_acc:0.5964994384125796
valid_acc:0.6392215568862275
train_avg_loss:1.033733327551575
valid_avg_loss:0.9463398832438598
 20%|███████████████▊                                                               | 1/5 [00:03<00:12,  3.23s/it]
epoch:1
train_acc:0.7048858105578435
valid_acc:0.6526946107784432
train_avg_loss:0.9151883647751875
valid_avg_loss:0.933617778900844
 40%|███████████████████████████████▌                                               | 2/5 [00:05<00:08,  2.83s/it]
epoch:2
train_acc:0.791463871209285
valid_acc:0.6422155688622755
train_avg_loss:0.8048011283679175
valid_avg_loss:0.9401777177604277
 60%|███████████████████████████████████████████████▍                               | 3/5 [00:08<00:05,  2.68s/it]
epoch:3
train_acc:0.863253463122426
valid_acc:0.6354790419161677
train_avg_loss:0.7067855492598396
valid_avg_loss:0.9559657252880175
 80%|███████████████████████████████████████████████████████████████▏               | 4/5 [00:10<00:02,  2.60s/it]
epoch:4
train_acc:0.9126731561213028
valid_acc:0.6317365269461078
train_avg_loss:0.6240185820416551
valid_avg_loss:0.9802028477414331
100%|███████████████████████████████████████████████████████████████████████████████| 5/5 [00:13<00:00,  2.67s/it]